In [1]:
import pandas as pd

In [2]:
# Carregar a base de óbitos
obitos_df = pd.read_csv("Agregados_por_distritos_obitos_BR.csv", encoding="latin1", sep=";")

In [3]:
# Carregar a base com área dos distritos
basico_df = pd.read_csv("Agregados_por_distritos_basico_BR.csv", encoding="latin1", sep=";")

In [4]:
# Remover espaços extras dos nomes das colunas
obitos_df.columns = obitos_df.columns.str.strip()
basico_df.columns = basico_df.columns.str.strip()

In [5]:
# Renomear colunas de interesse
basico_df.rename(columns={'v0001': 'POP_TOTAL'}, inplace=True)

In [6]:
obitos_df['V01224'] = pd.to_numeric(obitos_df['V01224'], errors='coerce')
obitos_df['V01225'] = pd.to_numeric(obitos_df['V01225'], errors='coerce')
obitos_df['PCT_DOMICILIOS_COM_OBITO'] = (obitos_df['V01224'] / (obitos_df['V01224'] + obitos_df['V01225'])) * 100

In [7]:
# Criar coluna M >= 60 ANOS como soma dos óbitos masculinos 60-69 anos (V01237) e >= 70 anos (V01238)
obitos_df['V01237'] = pd.to_numeric(obitos_df['V01237'], errors='coerce')
obitos_df['V01238'] = pd.to_numeric(obitos_df['V01238'], errors='coerce')
obitos_df['M >= 60 ANOS'] = obitos_df['V01237'] + obitos_df['V01238']

In [8]:
# Criar coluna M >= 60 ANOS como soma dos óbitos femininos 60-69 anos (V01248) e >= 70 anos (V01249)
obitos_df['V01248'] = pd.to_numeric(obitos_df['V01248'], errors='coerce')
obitos_df['V01249'] = pd.to_numeric(obitos_df['V01249'], errors='coerce')
obitos_df['F >= 60 ANOS'] = obitos_df['V01248'] + obitos_df['V01249']

In [9]:
# Criar coluna OBITOS como soma dos óbitos masculinos (V01224) e femininos (V01225)
obitos_df['V01226'] = pd.to_numeric(obitos_df['V01226'], errors='coerce')
obitos_df['V01227'] = pd.to_numeric(obitos_df['V01227'], errors='coerce')
obitos_df['OBITOS TOTAIS (M+F)'] = obitos_df['V01226'] + obitos_df['V01227']

In [10]:
# Criar coluna de óbitos M + F >= 60 anos em 2019
obitos_df['V01288'] = pd.to_numeric(obitos_df['V01288'], errors='coerce')
obitos_df['V01292'] = pd.to_numeric(obitos_df['V01292'], errors='coerce')
obitos_df['2019 (M+F >= 60 anos)'] = obitos_df['V01288'] + obitos_df['V01292']

In [11]:
# Criar coluna de óbitos M + F >= 60 anos em 2020
obitos_df['V01296'] = pd.to_numeric(obitos_df['V01296'], errors='coerce')
obitos_df['V01300'] = pd.to_numeric(obitos_df['V01300'], errors='coerce')
obitos_df['2020 (M+F >= 60 anos)'] = obitos_df['V01296'] + obitos_df['V01300']

In [12]:
# Criar coluna de óbitos M + F >= 60 anos em 2021
obitos_df['V01304'] = pd.to_numeric(obitos_df['V01304'], errors='coerce')
obitos_df['V01308'] = pd.to_numeric(obitos_df['V01308'], errors='coerce')
obitos_df['2021 (M+F >= 60 anos)'] = obitos_df['V01304'] + obitos_df['V01308']

In [13]:
# Criar coluna de óbitos M + F >= 60 anos em 2022
obitos_df['V01312'] = pd.to_numeric(obitos_df['V01312'], errors='coerce')
obitos_df['V01316'] = pd.to_numeric(obitos_df['V01316'], errors='coerce')
obitos_df['2022 (M+F >= 60 anos)'] = obitos_df['V01312'] + obitos_df['V01316']

In [14]:
# Selecionar colunas relevantes de cada DataFrame
obitos_selecionados = obitos_df[[
    "CD_DIST", "NM_DIST", "M >= 60 ANOS", "F >= 60 ANOS", "OBITOS TOTAIS (M+F)",
    "PCT_DOMICILIOS_COM_OBITO", "2019 (M+F >= 60 anos)", "2020 (M+F >= 60 anos)",
    "2021 (M+F >= 60 anos)", '2022 (M+F >= 60 anos)'
]]
area_selecionada = basico_df[["CD_DIST", "AREA_KM2", "POP_TOTAL"]]

In [15]:
# Mesclar os dados
df_merged = pd.merge(obitos_selecionados, area_selecionada, on="CD_DIST", how="left")

In [16]:
# Converter colunas para valores numéricos
colunas_numericas = ['POP_TOTAL', 'AREA_KM2', 'M >= 60 ANOS', 'F >= 60 ANOS', 'OBITOS TOTAIS (M+F)']
for coluna in colunas_numericas:
    df_merged[coluna] = (
        df_merged[coluna]
        .astype(str)  # garantir que é string
        .str.replace('.', '', regex=False)  # remove milhar (ex: 1.000 -> 1000)
        .str.replace(',', '.', regex=False)  # troca vírgula por ponto decimal
    )
    df_merged[coluna] = pd.to_numeric(df_merged[coluna], errors='coerce')  # converte para float
df_merged['M >= 60 ANOS'] = df_merged['M >= 60 ANOS'] / 10
df_merged['F >= 60 ANOS'] = df_merged['F >= 60 ANOS'] / 10
df_merged['OBITOS TOTAIS (M+F)'] = df_merged['OBITOS TOTAIS (M+F)'] / 10

In [17]:
# Criar novas colunas derivadas
df_merged["DENSIDADE_DEMOGRAFICA"] = df_merged["POP_TOTAL"] / df_merged["AREA_KM2"]
df_merged["PCT_IDOSOS"] = ((df_merged["M >= 60 ANOS"] + df_merged["F >= 60 ANOS"]) / df_merged["POP_TOTAL"]) * 100
df_merged["OBITOS_PER_CAPITA"] = df_merged["OBITOS TOTAIS (M+F)"] / df_merged["POP_TOTAL"]

In [18]:
# Exportar resultado
df_merged.to_csv("obitos_normalizados.csv", index=False, encoding='utf-8')
print("Arquivo 'obitos_normalizados.csv' criado com sucesso.")

Arquivo 'obitos_normalizados.csv' criado com sucesso.
